In [7]:
from Vector_Utils import *
import re

In [8]:
prompt = "Methane from fracking alone gaurantees existential global warming"
document = Document("article.docx")
full_text = "\n".join([para.text for para in document.paragraphs])

In [9]:
prompt = llmd_prompt(prompt, full_text)

In [10]:
print("Prompt:", prompt)
emphasis_words = prompt.split()

Prompt: US fracking boom and methane emissions threaten global climate stability.


In [11]:
vectorizer = vectorizer()
prompt_vec = vectorizer.encode(prompt)

In [ ]:
paragraphs = document.paragraphs
sentence_enders = re.compile(r'(?<=[.!?])\s+')

for paragraph in paragraphs:
    highest = 0.4
    best = ()

    sentences = re.split(sentence_enders, paragraph.text)

    if not vectorizer.screen_paragraph(sentences, prompt_vec, threshold=0.4):
        make_small(paragraph)
        continue

    if len(sentences) > 3:
        print("SPLIT")
        for i in range((len(sentences) // 3) + 1):
            k = i * 3
            if k + 3 < len(sentences):
                sub_paragraph = " ".join(sentences[k:k+3])
            else:
                sub_paragraph = " ".join(sentences[k:])
            sub_highest = 0.4
            sub_best = ()
            clauses = clause_separator(sub_paragraph)
            for combination in ordered_combinations(clauses, round(len(clauses) * 0.8)):
                joined = " ".join(combination)
                vector = vectorizer.encode(joined)
                sim_score = vectorizer.similarity(vector, prompt_vec)
                if sim_score > sub_highest:
                    sub_highest = sim_score
                    sub_best = combination
                    best += sub_best
                    print(sub_best)
    else:
        clauses = clause_separator(paragraph.text)
        for combination in ordered_combinations(clauses, round(len(clauses) * 0.8)):
            joined = " ".join(combination)
            vector = vectorizer.encode(joined)
            sim_score = vectorizer.similarity(vector, prompt_vec)
            if sim_score > highest:
                highest = sim_score
                best = combination

    print(highest, best)
    underline_best_match_in_paragraph(paragraph, best, emphasis_words)

document.save("article_annotated.docx")

0.748071014881134
0.7562731504440308 ('the world\'s leading exporter of liquefied natural gas within five years provoked warnings that the American fracking boom could "end hope for climate stability."',)
0.6061127185821533
SPLIT
('84 times more potent than carbon dioxide over a 20-year period. Methane emissions, by some estimates, are responsible for about a quarter of human-caused global warming. "Science confirms that gas is a climate killer," Wenonah Hauter, executive director of the U.S.',)
(", said in a statement Friday, citing methane's planet-warming potential.",)
0.5 ('84 times more potent than carbon dioxide over a 20-year period. Methane emissions, by some estimates, are responsible for about a quarter of human-caused global warming. "Science confirms that gas is a climate killer," Wenonah Hauter, executive director of the U.S.', ", said in a statement Friday, citing methane's planet-warming potential.")
0.27010345458984375
0.4031077027320862
SPLIT
0.5 ()
0.45001542568206787